# Processing all tissues in MouseDev project

In [26]:
import os

import starfish
from starfish.image import ApplyTransform, Filter, LearnTransform, Segment
from starfish.spots import FindSpots, DecodeSpots, AssignTargets
from starfish.types import Axes, FunctionSource
%matplotlib inline

import numpy as np
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import pprint

from starfish import data, FieldOfView, Experiment
from starfish.types import Features, Axes
from starfish.util.plot import imshow_plane
test = os.getenv("TESTING") is not None



## Define iss pipeline

In [27]:
def iss_pipeline(fov, codebook):
    primary_image = fov.get_image(starfish.FieldOfView.PRIMARY_IMAGES)
    
    dots = primary_image.reduce({Axes.CH, Axes.ROUND}, func="max")
    
    # register the raw image
    learn_translation = LearnTransform.Translation(reference_stack=dots, axes=Axes.ROUND, upsampling=100)
    
    transforms_list = learn_translation.run(primary_image.reduce({Axes.CH, Axes.ZPLANE}, func="max"))
    warp = ApplyTransform.Warp()
    registered = warp.run(primary_image, transforms_list=transforms_list,  in_place=False, verbose=True)
    
    # filter raw data
    masking_radius = 15
    filt = Filter.WhiteTophat(masking_radius, is_volume=False)
    filtered = filt.run(registered, verbose=True, in_place=False)

    bd = FindSpots.BlobDetector(
        min_sigma=1,
        max_sigma=10,
        num_sigma=30,
        threshold=0.01,
        measurement_type='mean',
    )

    # detect spots using laplacian of gaussians approach
    dots_max = dots.reduce((Axes.ROUND, Axes.ZPLANE), func="max")
    # locate spots in a reference image
    spots = bd.run(reference_image=dots_max, image_stack=filtered)

    # decode the pixel traces using the codebook
    decoder = DecodeSpots.PerRoundMaxChannel(codebook=codebook)
    decoded = decoder.run(spots=spots)

    # segment cells
    seg = Segment.Watershed(
        nuclei_threshold=.16,
        input_threshold=.22,
        min_distance=57,
    )
    label_image = seg.run(primary_image, dots)

    # assign spots to cells
    ta = AssignTargets.Label()
    assigned = ta.run(label_image, decoded)

    return assigned, label_image

## Define function to process all of the fovs

In [28]:
# process all the fields of view, not just one
def process_experiment(experiment: starfish.Experiment):
    decoded_intensities = {}
    regions = {}
    for i, (name_, fov) in enumerate(experiment.items()):
        print(name_)
        decoded, segmentation_results = iss_pipeline(fov, experiment.codebook)
        decoded_intensities[name_] = decoded
        regions[name_] = segmentation_results
        if test and i == 1:
            # only run through 2 fovs for the test
            break
    return decoded_intensities, regions

# Path to experiment.json file

In [29]:
path = 'H:/'
folders_to_remove = ['misc','Seagate','System Volume Information', '$RECYCLE.BIN']
output = [dI for dI in os.listdir(path) if os.path.isdir(os.path.join(path,dI))]
output_cleaned = [x for x in output if x not in folders_to_remove]
output_cleaned

['02A',
 '02B',
 '02C',
 '02D',
 '02E',
 '06A',
 '06B',
 '06C',
 '06D',
 '06E',
 '10A',
 '10B',
 '10C',
 '10D',
 '10E',
 '14A',
 '14B',
 '14C',
 '14D',
 '18A',
 '18B',
 '18C']

In [31]:
for i, slide_tag in enumerate(output_cleaned): 
    file = 'H:/'+slide_tag+'/SpaceTX_format/experiment.json'
    experiment = Experiment.from_json(file)
    print(slide_tag)
    print(experiment)
    decoded_intensities, regions = process_experiment(experiment)
    def getList(dict): 
        return dict.keys() 
    FOVS =getList(decoded_intensities)
    FOVS = list(FOVS)
    appended_data = []

    for i, FOV in enumerate(FOVS):
        FOV_data = decoded_intensities[FOV].to_features_dataframe()
        appended_data.append(FOV_data)
    appended_data = pd.concat(appended_data)
    appended_data_non_nan = appended_data[appended_data.target != 'nan']
    appended_data_non_nan.to_csv('H:/'+slide_tag+'/mouseDev_'+slide_tag+'_metricDistance.csv', sep = ',')
    appended_data_non_nan.to_csv('H:/'+slide_tag+'/mouseDev_'+slide_tag+'_metricDistance_withNaNs.csv', sep = ',')

02A
<starfish.Experiment (FOVs=25)>
{
fov_000: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, r: 5, c: 4, x: 2000, y: 2000)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, r: 1, c: 1, x: 2000, y: 2000)>
fov_001: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, r: 5, c: 4, x: 2000, y: 2000)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, r: 1, c: 1, x: 2000, y: 2000)>
fov_002: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, r: 5, c: 4, x: 2000, y: 2000)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, r: 1, c: 1, x: 2000, y: 2000)>
fov_003: <starfish.FieldOfView>
  Primary Image: <slicedimage.TileSet (z: 1, r: 5, c: 4, x: 2000, y: 2000)>
  Auxiliary Images:
    nuclei: <slicedimage.TileSet (z: 1, r: 1, c: 1, x: 2000, y: 2000)>
  ...,
}
fov_000


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 38.63it/s]


fov_001


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.63it/s]


fov_002


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.64it/s]


fov_040


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 36.96it/s]


fov_041


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 59.79it/s]
C:\Users\chris.langseth\Anaconda3\envs\starfish\lib\site-packages\skimage\feature\register_translation.py:103: RuntimeWarning: invalid value encountered in cdouble_scalars
  (src_amp * target_amp)
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 34.75it/s]
C:\Users\chris.langseth\Anaconda3\envs\starfish\lib\site-packages\starfish\core\util\levels.py:147: RuntimeWarning: invalid value encountered in true_divide
  array /= array.max()
C:\Users\chris.langseth\Anaconda3\envs\starfish\lib\site-packages\starfish\core\imagestack\imagestack.py:185: RuntimeWarning: invalid value encountered in less
  if np.min(data) < 0 or np.max(data) > 1:
C:\Users\chris.langseth\Anaconda3\envs\starfish\lib\site-packages\starfish\core\imagestack\imagestack.py:185: RuntimeWarning: invalid value encountered in greater
  if np.min(data)